In [41]:
import sys, os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import glob
currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir) + "/src/"
sys.path.insert(0, parentdir) 
from utils import get_data, preprocess_data

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Demo for Model Fitting
## Step 1: Data Loading

Let's try to fit some data and see how it goes. First, let's load the data for a stochasticity variant of our choice.

In [42]:
variant = "R"
data = get_data(variant = variant)

# get the first participant
participant = list(data.keys())[1]

data[participant] is a list of game data. We can take a look at one of these games.
- 'actions' contains a sequence of actions (True if left, False if right)
- 'baseline' contains the expected reward from a random policy on the boards
- 'boards' contains the face-value of the boards visible to the participant at every step (for example, in the volatility case this changes every step)
- 'p' contains the stochasticity level
- 'tuplepath' is the path the participant took, in a tuple
- 'total' is the total reward
- 'trials' contains the reaction times for each of the trials
- many other fields that aren't used much in the modeling part

In [43]:

print("Boards:\n", np.array(data[participant][0].boards)) # this is a list, but we just show the first board for now
print("Actions: ", data[participant][0].actions)
print("Tuplepath: ", data[participant][0].tuplepath)

stochasticity_level = data[participant][0].p
print("Stochasticity level: ", stochasticity_level)


Boards:
 [[[0 0 0 0 0 0 0 0]
  [2 5 0 0 0 0 0 0]
  [6 4 9 0 0 0 0 0]
  [3 1 3 8 0 0 0 0]
  [1 2 5 1 9 0 0 0]
  [6 5 8 4 3 7 0 0]
  [5 2 4 3 4 2 2 0]
  [6 1 3 5 3 4 4 8]]

 [[0 0 0 0 0 0 0 0]
  [2 9 0 0 0 0 0 0]
  [6 4 9 0 0 0 0 0]
  [3 1 3 8 0 0 0 0]
  [1 2 5 1 9 0 0 0]
  [6 5 8 4 3 7 0 0]
  [5 2 4 3 4 2 2 0]
  [6 1 3 5 3 4 4 8]]

 [[0 0 0 0 0 0 0 0]
  [2 9 0 0 0 0 0 0]
  [6 4 3 0 0 0 0 0]
  [3 1 3 8 0 0 0 0]
  [1 2 5 1 9 0 0 0]
  [6 5 8 4 3 7 0 0]
  [5 2 4 3 4 2 2 0]
  [6 1 3 5 3 4 4 8]]

 [[0 0 0 0 0 0 0 0]
  [2 9 0 0 0 0 0 0]
  [6 4 3 0 0 0 0 0]
  [3 1 3 8 0 0 0 0]
  [1 2 5 1 9 0 0 0]
  [6 5 8 4 3 7 0 0]
  [5 2 4 3 4 2 2 0]
  [6 1 3 5 3 4 4 8]]

 [[0 0 0 0 0 0 0 0]
  [2 9 0 0 0 0 0 0]
  [6 4 3 0 0 0 0 0]
  [3 1 3 8 0 0 0 0]
  [1 2 5 1 3 0 0 0]
  [6 5 8 4 3 7 0 0]
  [5 2 4 3 4 2 2 0]
  [6 1 3 5 3 4 4 8]]

 [[0 0 0 0 0 0 0 0]
  [2 9 0 0 0 0 0 0]
  [6 4 3 0 0 0 0 0]
  [3 1 3 8 0 0 0 0]
  [1 2 5 1 3 0 0 0]
  [6 5 8 4 3 4 0 0]
  [5 2 4 3 4 2 2 0]
  [6 1 3 5 3 4 4 8]]

 [[0 0 0 0 0 0 0 0]

We then preprocess the data from the participant. What we do is basically use the `xr.DataArray` object to combine these into a neat
multi-dimensional array that splits by conditions, games, trials, rows, columns. It basically just makes the data easier to organize, 
and makes it easier for us to read the data.

In [44]:
processed_data = preprocess_data(data[participant])

# so for example, the boards for the game we saw earlier can be accessed by
# selecting for the condition == stochasticity_level
print("boards:\n", processed_data.boards.sel(conditions = stochasticity_level, games = 0).values)

# and similarly we can access the actions they took - they should match with what we saw above
print("choose left: ", processed_data.choose_left.sel(conditions = stochasticity_level, games = 0).values)

# and also the paths taken
print("paths:", processed_data.paths.sel(conditions = stochasticity_level, games = 0).values.tolist())

boards:
 [[[0 0 0 0 0 0 0 0]
  [2 5 0 0 0 0 0 0]
  [6 4 9 0 0 0 0 0]
  [3 1 3 8 0 0 0 0]
  [1 2 5 1 9 0 0 0]
  [6 5 8 4 3 7 0 0]
  [5 2 4 3 4 2 2 0]
  [6 1 3 5 3 4 4 8]]

 [[0 0 0 0 0 0 0 0]
  [2 9 0 0 0 0 0 0]
  [6 4 9 0 0 0 0 0]
  [3 1 3 8 0 0 0 0]
  [1 2 5 1 9 0 0 0]
  [6 5 8 4 3 7 0 0]
  [5 2 4 3 4 2 2 0]
  [6 1 3 5 3 4 4 8]]

 [[0 0 0 0 0 0 0 0]
  [2 9 0 0 0 0 0 0]
  [6 4 3 0 0 0 0 0]
  [3 1 3 8 0 0 0 0]
  [1 2 5 1 9 0 0 0]
  [6 5 8 4 3 7 0 0]
  [5 2 4 3 4 2 2 0]
  [6 1 3 5 3 4 4 8]]

 [[0 0 0 0 0 0 0 0]
  [2 9 0 0 0 0 0 0]
  [6 4 3 0 0 0 0 0]
  [3 1 3 8 0 0 0 0]
  [1 2 5 1 9 0 0 0]
  [6 5 8 4 3 7 0 0]
  [5 2 4 3 4 2 2 0]
  [6 1 3 5 3 4 4 8]]

 [[0 0 0 0 0 0 0 0]
  [2 9 0 0 0 0 0 0]
  [6 4 3 0 0 0 0 0]
  [3 1 3 8 0 0 0 0]
  [1 2 5 1 3 0 0 0]
  [6 5 8 4 3 7 0 0]
  [5 2 4 3 4 2 2 0]
  [6 1 3 5 3 4 4 8]]

 [[0 0 0 0 0 0 0 0]
  [2 9 0 0 0 0 0 0]
  [6 4 3 0 0 0 0 0]
  [3 1 3 8 0 0 0 0]
  [1 2 5 1 3 0 0 0]
  [6 5 8 4 3 4 0 0]
  [5 2 4 3 4 2 2 0]
  [6 1 3 5 3 4 4 8]]

 [[0 0 0 0 0 0 0 0]

`pov_array` basically takes the "perspective" of the player. You can see for yourself that it matches the path the participant actually took and basically just makes `[0, 0]` the place where the participant "currently" is.

In [45]:
print("pov_array:\n", processed_data.pov_array.sel(conditions = stochasticity_level, games = 0).values)

pov_array:
 [[[0 0 0 0 0 0 0 0]
  [2 5 0 0 0 0 0 0]
  [6 4 9 0 0 0 0 0]
  [3 1 3 8 0 0 0 0]
  [1 2 5 1 9 0 0 0]
  [6 5 8 4 3 7 0 0]
  [5 2 4 3 4 2 2 0]
  [6 1 3 5 3 4 4 8]]

 [[9 0 0 0 0 0 0 0]
  [4 9 0 0 0 0 0 0]
  [1 3 8 0 0 0 0 0]
  [2 5 1 9 0 0 0 0]
  [5 8 4 3 7 0 0 0]
  [2 4 3 4 2 2 0 0]
  [1 3 5 3 4 4 8 0]
  [0 0 0 0 0 0 0 0]]

 [[3 0 0 0 0 0 0 0]
  [3 8 0 0 0 0 0 0]
  [5 1 9 0 0 0 0 0]
  [8 4 3 7 0 0 0 0]
  [4 3 4 2 2 0 0 0]
  [3 5 3 4 4 8 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]]

 [[8 0 0 0 0 0 0 0]
  [1 9 0 0 0 0 0 0]
  [4 3 7 0 0 0 0 0]
  [3 4 2 2 0 0 0 0]
  [5 3 4 4 8 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]]

 [[3 0 0 0 0 0 0 0]
  [3 7 0 0 0 0 0 0]
  [4 2 2 0 0 0 0 0]
  [3 4 4 8 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]]

 [[4 0 0 0 0 0 0 0]
  [2 2 0 0 0 0 0 0]
  [4 4 8 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]]

 [[1 0 0 0 0 0 0

## Step 2: Model Setup

In [46]:
from modeling import Model
from modelfilters import filter_depth
from modelvalues import value_path

The main thing to know here is that the model will take some parameters `params` and the boards `pov_array` and then make a prediction about the probability the player moves left at each step.

In [47]:
params = {
    "lapse": 0, 
    "condition_inv_temp_0": 10,
    "condition_inv_temp_1": 10,
    "condition_inv_temp_2": 10,
    "condition_inv_temp_3": 10,
    "condition_inv_temp_4": 10,
}
model = Model("policy_compress", filter_fn = filter_depth, value_fn = value_path, variant = variant)

In [48]:
# filter depth filters all the rows beyond row + depth
filtered_pov_array = model.filter_fn(processed_data.pov_array).sel(filter_params = 2, conditions = stochasticity_level, games = 0).transpose("trials", "rows",  "cols")
print(filtered_pov_array.values)

[[[0 0 0 0 0 0 0 0]
  [2 5 0 0 0 0 0 0]
  [6 4 9 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]]

 [[9 0 0 0 0 0 0 0]
  [4 9 0 0 0 0 0 0]
  [1 3 8 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]]

 [[3 0 0 0 0 0 0 0]
  [3 8 0 0 0 0 0 0]
  [5 1 9 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]]

 [[8 0 0 0 0 0 0 0]
  [1 9 0 0 0 0 0 0]
  [4 3 7 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]]

 [[3 0 0 0 0 0 0 0]
  [3 7 0 0 0 0 0 0]
  [4 2 2 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]]

 [[4 0 0 0 0 0 0 0]
  [2 2 0 0 0 0 0 0]
  [4 4 8 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]]

 [[1 0 0 0 0 0 0 0]
  [4 4 0

In [49]:
# tells us the value left - value right at each step
model.value_fn(filtered_pov_array)

<xarray.DataArray (trials: 7)> Size: 56B
array([ -6., -10.,  -9., -11.,  -2.,  -4.,   0.])
Coordinates:
    conditions     float64 8B 1.0
    filter_params  int64 8B 2
Dimensions without coordinates: trials

In [50]:
# tells us the probability of moving left at each step
p_left = model.get_prob_left(params, processed_data.pov_array)
for depth in range(8): 
    print(f"Predicted p_left (for depth {depth}): ", np.round(p_left.sel(conditions = stochasticity_level, games = 0, filter_params = depth).values, 2))
print("Actual p_left: ", processed_data.choose_left.sel(conditions = stochasticity_level, games = 0).values.astype(float))

Predicted p_left (for depth 0):  [0.5 0.5 0.5 0.5 0.5 0.5 0.5]
Predicted p_left (for depth 1):  [0.  0.  0.  0.  0.  0.5 0.5]
Predicted p_left (for depth 2):  [0.  0.  0.  0.  0.  0.  0.5]
Predicted p_left (for depth 3):  [0.  0.  0.  0.  0.  0.  0.5]
Predicted p_left (for depth 4):  [0.  0.  0.  0.  0.  0.  0.5]
Predicted p_left (for depth 5):  [0.  0.  0.  0.  0.  0.  0.5]
Predicted p_left (for depth 6):  [0.  0.  0.  0.  0.  0.  0.5]
Predicted p_left (for depth 7):  [0.  0.  0.  0.  0.  0.  0.5]
Actual p_left:  [0. 0. 0. 0. 0. 1. 0.]


/Users/jordanlei/miniconda3/lib/python3.13/site-packages/xarray/computation/apply_ufunc.py:821: RuntimeWarning: overflow encountered in exp
  result_data = func(*input_data)


Let's show an example with maybe more realistic parameters (the previous ones are helpful for illustration but lead to extreme log likelihoods, like -inf): 

In [51]:
params = {
    "lapse": 0, 
    "condition_inv_temp_0": -1,
    "condition_inv_temp_1": -1,
    "condition_inv_temp_2": -1,
    "condition_inv_temp_3": -1,
    "condition_inv_temp_4": -1,
}
model = Model("policy_compress", filter_fn = filter_depth, value_fn = value_path, variant = variant)
p_left = model.get_prob_left(params, processed_data.pov_array)
print("Predicted p_left: ", np.round(p_left.sel(conditions = stochasticity_level, games = 0).values, 2))
print("Actual p_left: ", processed_data.choose_left.sel(conditions = stochasticity_level, games = 0).values.astype(float))
log_likelihood = processed_data.choose_left * np.log(p_left) + (1 - processed_data.choose_left) * np.log(1 - p_left)
ll = log_likelihood.sum(["games", "trials", "conditions"])

for depth in range(8): 
    print(f"Predicted LL (for depth {depth}): ", ll.sel(filter_params = depth).values)

Predicted p_left:  [[0.5  0.5  0.5  0.5  0.5  0.5  0.5 ]
 [0.25 0.14 0.14 0.05 0.19 0.5  0.5 ]
 [0.1  0.02 0.04 0.02 0.32 0.19 0.5 ]
 [0.02 0.01 0.05 0.04 0.1  0.19 0.5 ]
 [0.   0.01 0.1  0.01 0.1  0.19 0.5 ]
 [0.   0.02 0.04 0.01 0.1  0.19 0.5 ]
 [0.01 0.01 0.04 0.01 0.1  0.19 0.5 ]
 [0.   0.01 0.04 0.01 0.1  0.19 0.5 ]]
Actual p_left:  [0. 0. 0. 0. 0. 1. 0.]
Predicted LL (for depth 0):  -727.8045395879425
Predicted LL (for depth 1):  -409.44717171898117
Predicted LL (for depth 2):  -431.48334331540116
Predicted LL (for depth 3):  -449.15525185193246
Predicted LL (for depth 4):  -468.6079825808091
Predicted LL (for depth 5):  -468.8914306689783
Predicted LL (for depth 6):  -469.4176034360893
Predicted LL (for depth 7):  -473.07556204147704


The `select_best_filter_params` finds the best filter parameter(s) given the other parameters of the model.

In [52]:
best_log_likelihood, best_filter_params = model.select_best_filter_params(params, processed_data.pov_array, processed_data.choose_left)
print("Maximum likelihood parameters: ", best_filter_params)
print("Maximum likelihood log likelihood: ", best_log_likelihood)

Maximum likelihood parameters:  {'global': np.int64(1)}
Maximum likelihood log likelihood:  -409.44717171898117


In [53]:
params = {
    "lapse": 0, 
    "condition_inv_temp_0": -1,
    "condition_inv_temp_1": -1,
    "condition_inv_temp_2": -1,
    "condition_inv_temp_3": -1,
    "condition_inv_temp_4": -1,
}
model.fit(params, data[participant])

/Users/jordanlei/miniconda3/lib/python3.13/site-packages/xarray/computation/apply_ufunc.py:821: RuntimeWarning: overflow encountered in exp
  result_data = func(*input_data)
/Users/jordanlei/miniconda3/lib/python3.13/site-packages/xarray/computation/apply_ufunc.py:821: RuntimeWarning: divide by zero encountered in log
  result_data = func(*input_data)
/Users/jordanlei/miniconda3/lib/python3.13/site-packages/xarray/computation/apply_ufunc.py:821: RuntimeWarning: overflow encountered in exp
  result_data = func(*input_data)
/Users/jordanlei/miniconda3/lib/python3.13/site-packages/xarray/computation/apply_ufunc.py:821: RuntimeWarning: overflow encountered in exp
  result_data = func(*input_data)
/Users/jordanlei/miniconda3/lib/python3.13/site-packages/xarray/computation/apply_ufunc.py:821: RuntimeWarning: divide by zero encountered in log
  result_data = func(*input_data)
/Users/jordanlei/miniconda3/lib/python3.13/site-packages/xarray/computation/apply_ufunc.py:821: RuntimeWarning: overfl

{'lapse': np.float64(0.029638492754376647),
 'condition_inv_temp_0': np.float64(0.33564195474120095),
 'condition_inv_temp_1': np.float64(0.19417402971316822),
 'condition_inv_temp_2': np.float64(0.3343172283565467),
 'condition_inv_temp_3': np.float64(-0.15110690508645783),
 'condition_inv_temp_4': np.float64(-0.09055415906624341),
 'filter_params': {'global': np.int64(1)},
 'nll': 316.9324035510777,
 'model': 'policy_compress.filter_depth.value_path'}

Now let's try seeing what happens when we fit for a `filter_adaptation` model

In [60]:
params = {
    "lapse": 0, 
    "inv_temp": 0,
}
model = Model("filter_adapt", filter_fn = filter_depth, value_fn = value_path, variant = variant)
p_left = model.get_prob_left(params, processed_data.pov_array)
print("Predicted p_left: ", np.round(p_left.sel(conditions = stochasticity_level, games = 0).values, 2))
print("Actual p_left: ", processed_data.choose_left.sel(conditions = stochasticity_level, games = 0).values.astype(float))
log_likelihood = processed_data.choose_left * np.log(p_left) + (1 - processed_data.choose_left) * np.log(1 - p_left)
ll = log_likelihood.sum(["games", "trials"])

for depth in range(8): 
    print(f"Predicted LL (for depth {depth}): ", ll.sel(filter_params = depth).values)

Predicted p_left:  [[0.5  0.5  0.5  0.5  0.5  0.5  0.5 ]
 [0.05 0.01 0.01 0.   0.02 0.5  0.5 ]
 [0.   0.   0.   0.   0.12 0.02 0.5 ]
 [0.   0.   0.   0.   0.   0.02 0.5 ]
 [0.   0.   0.   0.   0.   0.02 0.5 ]
 [0.   0.   0.   0.   0.   0.02 0.5 ]
 [0.   0.   0.   0.   0.   0.02 0.5 ]
 [0.   0.   0.   0.   0.   0.02 0.5 ]]
Actual p_left:  [0. 0. 0. 0. 0. 1. 0.]
Predicted LL (for depth 0):  [-145.56090792 -145.56090792 -145.56090792 -145.56090792 -145.56090792]
Predicted LL (for depth 1):  [-60.90542857 -64.50837153 -45.48596371 -77.08773333 -73.79684847]
Predicted LL (for depth 2):  [ -89.02978793  -86.66587571  -57.78360413 -105.2402446  -105.03719432]
Predicted LL (for depth 3):  [ -88.44611283  -88.7327923   -74.26224726 -125.8520332  -129.60857489]
Predicted LL (for depth 4):  [ -94.37728897  -97.86325534  -83.65104912 -138.03561789 -153.26932495]
Predicted LL (for depth 5):  [-100.71693548 -100.31325191  -81.10761469 -138.69346333 -146.43284614]
Predicted LL (for depth 6):  [-106.6

In [61]:
best_log_likelihood, best_filter_params = model.select_best_filter_params(params, processed_data.pov_array, processed_data.choose_left)
print("Maximum likelihood parameters: ", best_filter_params)
print("Maximum likelihood log likelihood: ", best_log_likelihood)

Maximum likelihood parameters:  {np.float64(0.0): np.int64(1), np.float64(0.25): np.int64(1), np.float64(0.5): np.int64(1), np.float64(0.75): np.int64(1), np.float64(1.0): np.int64(1)}
Maximum likelihood log likelihood:  <xarray.DataArray ()> Size: 8B
array(-321.7843456)


In [62]:
model.fit(params, data[participant])

{'lapse': np.float64(0.021636282094428805),
 'inv_temp': np.float64(0.05355772999613185),
 'filter_params': {np.float64(0.0): np.int64(1),
  np.float64(0.25): np.int64(1),
  np.float64(0.5): np.int64(1),
  np.float64(0.75): np.int64(1),
  np.float64(1.0): np.int64(1)},
 'nll': 320.4343980200837,
 'model': 'filter_adapt.filter_depth.value_path'}